<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 📊 PySpark: החלפת ערכי עמודות ב-DataFrame
[PySpark Replace Column Values in DataFrame](https://sparkbyexamples.com/pyspark/pyspark-replace-column-values/)
## תוכן עניינים
1. [החלפת ערכי עמודת מחרוזות](#section1)
2. [החלפת ערכים בתנאי](#section2)
3. [החלפת ערכים באמצעות מילון (Dictionary)](#section3)
4. [החלפת תו בתו](#section4)
5. [החלפת עמודה בערך מעמודה אחרת](#section5)
6. [החלפת כל העמודות או מספר עמודות](#section6)
7. [שימוש בפונקציה overlay()](#section7)
8. [דוגמה מלאה](#section8)

---

## 🎯 מבוא

במדריך זה נלמד כיצד להחליף ערכים בעמודות של PySpark DataFrame באמצעות פונקציות SQL כמו:
- `regexp_replace()` - החלפה באמצעות ביטויים רגולריים
- `translate()` - החלפת תווים בודדים
- `overlay()` - החלפת חלק ממחרוזת

נכסה דוגמאות מעשיות להחלפת חלק ממחרוזת, החלפת כל העמודות, שינוי ערכים בתנאי, החלפת ערכים ממילון Python, והחלפת ערך עמודה מעמודה אחרת ב-DataFrame.

---

## 🔧 הכנת הסביבה וטעינת נתונים

ראשית, נייצר DataFrame לדוגמה עם כתובות שנשתמש בהן לכל הדוגמאות במדריך.

In [ ]:
# ייבוא הספריות הנדרשות
# Create sample Data
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

address = [
    (1, "14851 Jeffrey Rd", "DE"),
    (2, "43421 Margarita St", "NY"),
    (3, "13111 Siemon Ave", "CA")
]

df = spark.createDataFrame(address, ["id", "address", "state"])
df.show()

---

<a id="section1"></a>
## 1️⃣ החלפת ערכי עמודת מחרוזות (Replace String Column Values)

### 📝 הסבר

באמצעות הפונקציה `regexp_replace()` של PySpark SQL ניתן להחליף ערך עמודה במחרוזת אחרת או בתת-מחרוזת.

הפונקציה `regexp_replace()` משתמשת ב-**Java regex** להתאמה. אם הביטוי הרגולרי לא תואם, הפונקציה מחזירה מחרוזת רקה.

בדוגמה הבאה, נחליף את שם הרחוב `Rd` במילה `Road` בעמודת `address`:

In [ ]:
# החלפת חלק ממחרוזת במחרוזת אחרת
from pyspark.sql.functions import regexp_replace

df.withColumn(
    'address', 
    regexp_replace(df.address, 'Rd', 'Road')
).show(truncate=False)

---

<a id="section2"></a>
## 2️⃣ החלפת ערכי עמודה בתנאי (Replace Column Values Conditionally)

### 📝 הסבר

בדוגמה הקודמת החלפנו רק את `Rd` ב-`Road`, אך לא החלפנו את `St` ו-`Ave`. 

כעת נראה כיצד להחליף ערכי עמודה **בתנאי** ב-PySpark DataFrame באמצעות פונקציות:
- `when()` - תנאי IF
- `otherwise()` - תנאי ELSE
- SQL condition functions

In [ ]:
# החלפת ערכי עמודת מחרוזת בתנאי
from pyspark.sql.functions import when

df.withColumn(
    'address',
    when(df.address.endswith('Rd'), regexp_replace(df.address, 'Rd', 'Road'))
    .when(df.address.endswith('St'), regexp_replace(df.address, 'St', 'Street'))
    .when(df.address.endswith('Ave'), regexp_replace(df.address, 'Ave', 'Avenue'))
    .otherwise(df.address)
).show(truncate=False)

---

<a id="section3"></a>
## 3️⃣ החלפת ערכי עמודה עם מילון - Dictionary (map)

### 📝 הסבר

ניתן גם להחליף ערכי עמודה מ-**Python dictionary (map)**.

בדוגמה הבאה, נחליף את ערך המחרוזת בעמודת `state` עם השם המלא המקוצר מהמילון (key-value pair).

כדי לעשות זאת, נשתמש ב-**PySpark map() transformation כדי לעבור על כל שורה** ב-DataFrame:

In [ ]:
# החלפת ערכים ממילון (Dictionary)
stateDic = {
    'CA': 'California',
    'NY': 'New York',
    'DE': 'Delaware'
}

df2 = df.rdd.map(
    lambda x: (x.id, x.address, stateDic[x.state])
).toDF(["id", "address", "state"])

df2.show()

---

<a id="section4"></a>
## 4️⃣ החלפת תו בתו (Replace Column Value Character by Character)

### 📝 הסבר

באמצעות הפונקציה `translate()` ניתן **להחליף תו בתו** בערך עמודת DataFrame.

בדוגמה הבאה, כל תו `1` מוחלף ב-`A`, `2` מוחלף ב-`B`, ו-`3` מוחלף ב-`C` בעמודת `address`:

In [ ]:
# שימוש ב-translate להחלפת תווים בודדים
from pyspark.sql.functions import translate

df.withColumn(
    'address', 
    translate(df.address, '123', 'ABC')
).show(truncate=False)

---

<a id="section5"></a>
## 5️⃣ החלפת עמודה בערך מעמודה אחרת (Replace Column with Another Column Value)

### 📝 הסבר

באמצעות `expr()` ו-`regexp_replace()` ניתן **להחליף ערך עמודה בערך מעמודת DataFrame אחרת**.

בדוגמה הבאה, נתאים את הערך מעמודה `col2` בעמודה `col1` ונחליף אותו ב-`col3` כדי ליצור עמודה חדשה `new_column`.

משתמשים ב-`expr()` כדי לספק ביטויים דמויי SQL, והוא משמש להתייחסות לעמודה אחרת בביצוע פעולות:

In [ ]:
# החלפת עמודה בערך מעמודה אחרת
from pyspark.sql.functions import expr

df = spark.createDataFrame(
    [('ABCDE_XYZ', 'XYZ', 'FGH'),('111_222_333', '222', '888')],
    ['col1', 'col2', 'col3']
)
df.show()

In [ ]:
df.withColumn(
    'new_column',
    expr("regexp_replace(col1, col2, col3)")
).alias("replaced_value").show()

---

<a id="section6"></a>
## 6️⃣ החלפת כל העמודות או מספר עמודות (Replace All or Multiple Column Values)

### 📝 הסבר

אם ברצונך להחליף ערכים בכל העמודות או במספר עמודות נבחרות ב-DataFrame, עיין במדריכים הבאים:

- 📘 [כיצד להחליף NULL/None values בכל העמודות ב-PySpark](https://sparkbyexamples.com/pyspark/pyspark-fillna-fill-replace-null-values/)
- 📘 [כיצד להחליף מחרוזת ריקה עם NULL/None value](https://sparkbyexamples.com/pyspark/pyspark-replace-empty-value-with-none/)

---

<a id="section7"></a>
## 7️⃣ שימוש בפונקציה overlay()

### 📝 הסבר

הפונקציה `overlay()` מאפשרת **להחליף ערך עמודת מחרוזת בערך מחרוזת מעמודה אחרת**.

In [ ]:
# דוגמה לשימוש ב-overlay
from pyspark.sql.functions import overlay

df = spark.createDataFrame(
    [('ABCDE_XYZ', 'FGH'),('111_222_333', '888')],
    ['col1', 'col2']
)
df.show()

In [ ]:
df = df.withColumn('overlayed',
    overlay('col1', 'col2', 7)
)
df.show()

---

<a id="section8"></a>
## 8️⃣ דוגמה מלאה - כל הקוד ביחד

### 📝 הסבר

להלן דוגמה מלאה הכוללת את כל השיטות שלמדנו במדריך זה:

In [ ]:
# קוד מלא - כל הדוגמאות ביחד
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

address = [
    (1, "14851 Jeffrey Rd", "DE"),
    (2, "43421 Margarita St", "NY"),
    (3, "13111 Siemon Ave", "CA")
]

df = spark.createDataFrame(address, ["id", "address", "state"])
df.show()

In [ ]:
# החלפת מחרוזת פשוטה
from pyspark.sql.functions import regexp_replace

df.withColumn(
    'address', 
    regexp_replace(df.address, 'Rd', 'Road')
).show(truncate=False)

In [ ]:
# החלפת מחרוזת בתנאי
from pyspark.sql.functions import when

df.withColumn(
    'address',
    when(df.address.endswith('Rd'), regexp_replace(df.address, 'Rd', 'Road'))
    .when(df.address.endswith('St'), regexp_replace(df.address, 'St', 'Street'))
    .when(df.address.endswith('Ave'), regexp_replace(df.address, 'Ave', 'Avenue'))
    .otherwise(df.address)
).show(truncate=False)

In [ ]:
# החלפת ערכים ממילון (Dictionary)
stateDic = {
    'CA': 'California',
    'NY': 'New York',
    'DE': 'Delaware'
}

df2 = df.rdd.map(
    lambda x: (x.id, x.address, stateDic[x.state])
).toDF(["id", "address", "state"])

df2.show()

In [ ]:
# שימוש ב-translate
from pyspark.sql.functions import translate

df.withColumn(
    'address', 
    translate(df.address, '123', 'ABC')
).show(truncate=False)

In [ ]:
# החלפת עמודה בערך מעמודה אחרת
from pyspark.sql.functions import expr

df = spark.createDataFrame(
    [('ABCDE_XYZ', 'XYZ', 'FGH')],
    ['col1', 'col2', 'col3']
)

df.withColumn(
    'new_column',
    expr("regexp_replace(col1, col2, col3)")
).alias("replaced_value").show()

In [ ]:
# Overlay
from pyspark.sql.functions import overlay

df = spark.createDataFrame(
    [('ABCDE_XYZ', 'FGH')],
    ['col1', 'col2']
)

df.select(
    overlay('col1', 'col2', 7).alias('overlayed')
).show()

---

## 🎓 סיכום

במדריך זה למדנו:

✅ הפונקציה `regexp_replace()` משמשת להחלפת מחרוזת בעמודת DataFrame עם ערך אחר

✅ הפונקציה `translate()` משמשת להחלפת תו בתו של ערכי עמודות

✅ הפונקציה `overlay()` משמשת לשכתוב מחרוזת עם מחרוזת עמודה אחרת ממיקום התחלה ומספר תווים

✅ בנוסף, למדנו כיצד להחליף ערכי עמודות ממילון (dictionary) באמצעות דוגמאות Python

**🎉 למידה מהנה!**

---

## 📚 מקורות ומאמרים נוספים

### מאמרים קשורים:
- [PySpark repartition() – Explained with Examples](https://sparkbyexamples.com/pyspark/pyspark-repartition/)
- [PySpark Replace Empty Value With None/null on DataFrame](https://sparkbyexamples.com/pyspark/pyspark-replace-empty-value-with-none/)
- [PySpark createOrReplaceTempView() Explained](https://sparkbyexamples.com/pyspark/pyspark-create-or-replace-temp-view/)
- [PySpark fillna() & fill() – Replace NULL/None Values](https://sparkbyexamples.com/pyspark/pyspark-fillna-fill-replace-null-values/)
- [PySpark repartition() vs partitionBy()](https://sparkbyexamples.com/pyspark/pyspark-repartition-vs-partitionby/)
- [PySpark Repartition() vs Coalesce()](https://sparkbyexamples.com/pyspark/pyspark-repartition-vs-coalesce/)
- [PySpark Replace Empty Value With None/null on DataFrame](https://sparkbyexamples.com/pyspark/pyspark-replace-empty-value-with-none/)
- [Pyspark Select Distinct Rows](https://sparkbyexamples.com/pyspark/pyspark-distinct-to-drop-duplicates/)
- [PySpark Get Number of Rows and Columns](https://sparkbyexamples.com/pyspark/pyspark-get-number-of-rows-and-columns/)

### מקורות חיצוניים:
- [Apache Spark Documentation](https://spark.apache.org/docs/latest/)
- [PySpark SQL Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html)
- [Handling Null/Empty Strings in PySpark](https://kb.databricks.com/data/null-empty-strings.html)

---

**© SparkByExamples.com - All rights reserved**

*מדריך זה תורגם ועובד לעברית למטרות לימוד*